In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# SED for Trappist-1
This notebook executes the lines of code presented in Filippazzo et al. (submitted to PASP). Below I generate and analyze an SED for the M8 dwarfs Trappist-1 and Gl 752B. Then I create a catalog of SEDs for analysis.

In [187]:
# Imports
from astropy import units as q
from bokeh.io import output_notebook
from bokeh.plotting import show
from pkg_resources import resource_filename
output_notebook()

Loading BokehJS ...

## Initialization
Import the `SED` class and create a Trappist-1 object. The parallax, radius and spectral type are pulled from Simbad when available.

In [188]:
from sedkit import SED
trap1 = SED(name='Trappist-1')

[sedkit] SED initialized
[sedkit] Setting name to 'Trappist-1'
[sedkit] 1 record for Trappist-1 found in Simbad.
[sedkit] Setting sky_coords to <SkyCoord (ICRS): (ra, dec) in deg
    (23.10815789, -5.04139917)>
[sedkit] There was a problem determining the interstellar reddening. Setting to 0. You can manually set this with the 'reddening' attribute.
[sedkit] Setting parallax to (<Quantity 80.2123 mas>, <Quantity 0.0716 mas>) with reference '2020yCat.1350....0G'
[sedkit] Setting distance to (<Quantity 12.47 pc>, <Quantity 0.01 pc>, <Quantity 0.01 pc>) with reference '2020yCat.1350....0G'
[sedkit] Setting interstellar reddening to 0.0 with reference '2018JOSS....3..695M'
[sedkit] Setting radius to 'None'
No valid data in the arrays
Could not evaluate the radius(spt) relation at (67.5, 0.5)
[sedkit] Setting radius to 'None'
[sedkit] Setting spectral_type to (67.5, 0.5, 'V', None, None) with reference '2000AJ....120.1085G'


## Photometry
Add photometry manually...

In [189]:
trap1.add_photometry('Johnson.V', 18.798, 0.082, ref='Costa et al. (2006)')
trap1.add_photometry('Cousins.R', 16.466, 0.065, ref='Costa et al. (2006)')
trap1.add_photometry('Cousins.I', 14.024, 0.115, ref='Costa et al. (2006)')

[sedkit] Setting Johnson.V photometry to 18.798 (0.082) with reference 'Costa et al. (2006)'
[sedkit] Setting Cousins.R photometry to 16.466 (0.065) with reference 'Costa et al. (2006)'
[sedkit] Setting Cousins.I photometry to 14.024 (0.115) with reference 'Costa et al. (2006)'


...and retrieve some from Simbad.

In [190]:
trap1.find_2MASS()
trap1.find_WISE()

[sedkit] 10 records found in '2MASS' using target name 'Trappist-1'
[sedkit] Setting 2MASS.J photometry to 11.354 (0.022) with reference 'II/246/out'
[sedkit] Setting 2MASS.H photometry to 10.718 (0.021) with reference 'II/246/out'
[sedkit] Setting 2MASS.Ks photometry to 10.296 (0.023) with reference 'II/246/out'
[sedkit] 46 records found in 'WISE' using target name 'Trappist-1'
[sedkit] Setting WISE.W1 photometry to 10.068 (0.023) with reference 'II/328/allwise'
[sedkit] Setting WISE.W2 photometry to 9.807 (0.020) with reference 'II/328/allwise'
[sedkit] Setting WISE.W3 photometry to 9.510 (0.044) with reference 'II/328/allwise'
[sedkit] Setting WISE.W4 photometry to 8.565 (nan) with reference 'II/328/allwise'


In [191]:
# Print the photometry table
trap1.photometry[['band', 'eff', 'app_magnitude', 'app_magnitude_unc', 'ref']].pprint()

   band        eff      app_magnitude app_magnitude_unc         ref        
                um                                                         
--------- ------------- ------------- ----------------- -------------------
Johnson.V 0.54638671875          18.8             0.082 Costa et al. (2006)
Cousins.R  0.6357421875         16.47             0.065 Costa et al. (2006)
Cousins.I 0.78271484375         14.02             0.115 Costa et al. (2006)
  2MASS.J  1.2353515625         11.35             0.022          II/246/out
  2MASS.H   1.662109375         10.72             0.021          II/246/out
 2MASS.Ks   2.158203125          10.3             0.023          II/246/out
  WISE.W1   3.353515625         10.07             0.023      II/328/allwise
  WISE.W2     4.6015625         9.805              0.02      II/328/allwise
  WISE.W3       11.5625          9.51             0.044      II/328/allwise
  WISE.W4      22.09375          8.56               nan      II/328/allwise


## Spectra

In [192]:
# Grab spectrum files
nir_spec = resource_filename('sedkit', 'data/Trappist-1_NIR.fits')

# Add the spectra (trimming edge effects off the optical one)
trap1.add_spectrum_file(nir_spec, name='NIR spectrum', wave_units=q.um, flux_units=q.W/q.m**2/q.um, ref='Filippazzo et al. (2015)')

# Print the table
trap1.spectra[['name', 'wave_min', 'wave_max', 'resolution', 'ref']].pprint()

[sedkit] Spectrum 'NIR spectrum' added with reference 'Filippazzo et al. (2015)'
    name       wave_min    wave_max  resolution           ref           
                  um          um                                        
------------ ------------ ---------- ---------- ------------------------
NIR spectrum 0.6533203125 2.56640625        563 Filippazzo et al. (2015)


## Other Data
Set the spectral type.

In [193]:
trap1.spectral_type = 'M8', '2006AJ....132.1234C'

[sedkit] Setting radius to 'None'
No valid data in the arrays
Could not evaluate the radius(spt) relation at (68.0, 0.5)
[sedkit] Setting radius to 'None'
[sedkit] Setting spectral_type to (68.0, 0.5, 'V', None, None) with reference '2006AJ....132.1234C'


Set the age.

In [194]:
trap1.age = 7.6*q.Gyr, 2.2*q.Gyr, '2017ApJ...845..110B'

[sedkit] Setting age to (<Quantity 7.6 Gyr>, <Quantity 2.2 Gyr>) with reference '2017ApJ...845..110B'


Set the radius.

In [195]:
trap1.radius = 0.116*q.R_sun, 0.003*q.R_sun, '2021PSJ.....2....1A'

[sedkit] Setting radius to (<Quantity 0.116 solRad>, <Quantity 0.003 solRad>) with reference '2021PSJ.....2....1A'


## Build SED

In [196]:
trap1.make_sed()
trap1.plot(integral=True)

[sedkit] Setting fbol to (<Quantity 1.19872967e-10 erg / (cm2 s)>, <Quantity 7.56413339e-13 erg / (cm2 s)>) with reference 'This Work'
[sedkit] Setting Lbol to (<Quantity 2.2303094e+30 erg / s>, <Quantity 1.45210105e+28 erg / s>) with reference 'This Work'
[sedkit] Setting Lbol_sun to (-3.235, 0.003) with reference 'This Work'
[sedkit] Setting mbol to (13.321, 0.007) with reference 'This Work'
[sedkit] Setting Mbol to (12.842, 0.007) with reference 'This Work'
[sedkit] Setting logg to 'None'
[sedkit] Setting logg to [5.28, 0.0] with reference 'DUSTY00'
[sedkit] Setting mass to 'None'
[sedkit] Setting mass to (<Quantity 0.09 solMass>, <Quantity 0. solMass>, <Quantity 0. solMass>) with reference '2013ApJS..208....9P'
[sedkit] Setting Teff to (<Quantity 2632 K>, <Quantity 34 K>) with reference 'This Work'


Figure(id='43374', ...)

## Fit the data

In [197]:
# Fit to infer spectral type
trap1.fit_spectral_type()

ValueError: unsupported pickle protocol: 5

In [ ]:
# Fit to infer fundamental params
from sedkit import BTSettl
trap1.fit_modelgrid(BTSettl())

## Inferring mass, surface gravity, radius, and Teff
Set the evolutionary model grid and optionally infer fundamental parameters.

In [198]:
trap1.evo_model = 'DUSTY00'

In [199]:
trap1.infer_logg(plot=True)

[sedkit] Setting logg to 'None'


[sedkit] Setting logg to [5.28, 0.0] with reference 'DUSTY00'


In [202]:
trap1.infer_mass(plot=True, mass_units=q.Mjup)

[sedkit] Setting mass to 'None'


[sedkit] Setting mass to (<Quantity 94.28089632 jupiterMass>, <Quantity 0. jupiterMass>, <Quantity 0. jupiterMass>) with reference '2013ApJS..208....9P'


## Results
The results of the SED calculations can be accessed anytime with the `results` property.

In [201]:
trap1.results

param,value,unc,units
object,object,object,object
name,Trappist-1,--,--
ra,23.10815788888889,--,deg
dec,-5.041399166666666,--,deg
age,7.6,2.2,Gyr
membership,--,--,--
distance,12.47,0.01,pc
parallax,80.2123,0.0716,mas
SpT,M8V,--,--
spectral_type,68.0,0.5,--


The `plot` method plots the SED, spectra, and photometry by default. The best fits and integral surface used to calculate `Lbol` can be included by setting their arguments to `True`. Click on the elements in the legend to toggle.

In [18]:
plt = trap1.plot(integral=True, best_fit=True)

Check out the references of all the data used in the SED with the `refs` property.

In [19]:
trap1.refs

{'parallax': '2018yCat.1345....0G',
 'distance': '2018yCat.1345....0G',
 'reddening': '2018JOSS....3..695M',
 'radius': '2021PSJ.....2....1A',
 'spectral_type': '2006AJ....132.1234C',
 'age': '2017ApJ...845..110B',
 'fbol': 'This Work',
 'Lbol': 'This Work',
 'Lbol_sun': 'This Work',
 'mbol': 'This Work',
 'Mbol': 'This Work',
 'logg': 'DUSTY00',
 'mass': '2013ApJS..208....9P',
 'Teff': 'This Work',
 'photometry': {'Johnson.V': 'Costa et al. (2006)',
  'Cousins.R': 'Costa et al. (2006)',
  'Cousins.I': 'Costa et al. (2006)',
  '2MASS.J': 'II/246/out',
  '2MASS.H': 'II/246/out',
  '2MASS.Ks': 'II/246/out',
  'WISE.W1': 'II/328/allwise',
  'WISE.W2': 'II/328/allwise',
  'WISE.W3': 'II/328/allwise',
  'WISE.W4': 'II/328/allwise'},
 'spectra': {'NIR spectrum': 'Filippazzo et al. (2015)'},
 'best_fit': {'SpeX Prism Library spec': '2014ASInC..11....7B',
  'BT-Settl spec': '2014IAUS..299..271A'}}

And look at all the best fit models with the `best_fit` property.

In [20]:
trap1.best_fit

{'SpeX Prism Library spec': {'spty': 'Opt:M8V',
  'filepath': '/Users/jfilippazzo/Documents/Modules/sedkit/sedkit/data/models/atmospheric/spexprismlibrary/spex-prism_VB10_20030919_BUR04B.txt.xml',
  'spectrum': array([[6.5798396e-01, 6.6016799e-01, 6.6234499e-01, ..., 2.5545900e+00,
          2.5575399e+00, 2.5605099e+00],
         [4.0704105e-16, 3.3004014e-16, 3.1254883e-16, ..., 1.4180143e-15,
          1.6574101e-15, 1.4649808e-15]], dtype=float32),
  'label': 'Opt:M8V',
  'SpT': 68.0,
  'gstat': 0.00250764636049802,
  'const': 9.709833028611168e-14,
  'wave_units': Unit("um"),
  'flux_units': Unit("erg / (Angstrom cm2 s)"),
  'fit_to': 'spec',
  'full_model': <sedkit.spectrum.Spectrum at 0x199110fa0>},
 'BT-Settl spec': {'alpha': 0.0,
  'filepath': '/Users/jfilippazzo/Documents/Modules/sedkit/sedkit/data/models/atmospheric/btsettl/lte026-5.5-0.0a+0.0.BT-NextGen.7.dat.xml',
  'label': '2600.0/5.5/0.0/0.0',
  'logg': 5.5,
  'meta': 0.0,
  'spectrum': array([[3.00001489e-01, 3.024716